#Связь бизнес-показателей и DS-метрик

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какой график (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
import scipy.stats as st
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

In [4]:
df = pd.read_csv('/content/drive/MyDrive/train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [5]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], 
                                                    stratify=df['cardio'],
                                                    random_state=0)

In [6]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
        
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]        

In [7]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [8]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)

In [9]:
# Функция для рассчета доверительного интервала
def mean_confidence_interval(data, confidence=0.95):
    _, right = st.t.interval(0.95, len(data)-1, loc=np.mean(data), scale=st.sem(data))
    deviance = right - np.mean(data)
    return np.mean(data), deviance

In [11]:
# Функция для проведения кроссвалидации и рассчета доверительного интервала метрики
def cross_val_conf_int(model, X_train, y_train, cv=16, scoring='roc_auc'):
    cv_scores = cross_val_score(model, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score, deviance = mean_confidence_interval(cv_scores)
    print('CV score is {}+-{}'.format(cv_score, deviance))

In [12]:
# Конвеер для логистической регрессии
logreg_classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [13]:
cross_val_conf_int(logreg_classifier, X_train, y_train, cv=16, scoring='roc_auc')

CV score is 0.7825870713544835+-0.004598369745713704


In [14]:
# Конвеер для случайного леса
randfor_classifier = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state=42,
                                         max_depth=8,
                                         min_samples_leaf=20,
                                         n_estimators=20)),
])

In [20]:
cross_val_conf_int(randfor_classifier, X_train, y_train, cv=16, scoring='roc_auc')

CV score is 0.7993347777709795+-0.004306028561452013


Вывод: при увеличении сложности алгоритма, метрика roc_auc растет

Создадим теперь сводную таблицу с метриками

In [18]:
columns = ['roc_auc', 'precision', 'recall', 'f1_score', 'Best Threshold']
index = ['logreg', 'random_forest']
metrics_table = pd.DataFrame(columns=columns, index=index)

In [23]:
def find_best_metrics(y_test, y_score, b=1):
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_score)
    fscore = (1+b**2)*(precision[:-10] * recall[:-10]) / (b**2*precision[:-10] + recall[:-10])
    roc_auc = roc_auc_score(y_true=y_test, y_score=y_score)
    ix = np.argmax(fscore)
    return roc_auc, precision[ix], recall[ix], fscore[ix], thresholds[ix]

In [24]:
models_dict = {'logreg': logreg_classifier, 'random_forest': randfor_classifier}
for key in models_dict:
    models_dict[key].fit(X_train, y_train)
    y_score = models_dict[key].predict_proba(X_test)[:, 1]
    metrics_table.loc[key] = find_best_metrics(y_test, y_score)
    
metrics_table

,roc_auc,precision,recall,f1_score,Best Threshold
logreg,0.784676,0.671638,0.808576,0.733773,0.417384
random_forest,0.798931,0.682206,0.813265,0.741993,0.392866


Вывод: Лучшие метрики у модели случайного леса. Чем сложнее алгоритм, тем более высокие метрики он будет показывать